In [3]:
import opensmile
import os
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

# Définir le chemin du dossier contenant les audios et le fichier des émotions
audio_folder = "dataset_clean/train/audios/"
emotion_file = "dataset_clean/train/train_sent_emo2.csv"

# Charger le dataframe contenant les émotions
df = pd.read_csv(emotion_file, sep=";", encoding="cp1252")

# Créer un dataframe avec les émotions et les énoncés
df_relevant = df[['Emotion', 'Utterance']].dropna()

# Initialiser le LabelEncoder pour convertir les étiquettes d'émotions en entiers
label_encoder = LabelEncoder()
label_encoder.fit(df_relevant['Emotion'].unique())  # Apprendre les étiquettes d'émotions

# Créer un objet OpenSMILE avec une configuration valide
smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.ComParE_2016,  # Configuration pour ComParE_2016
    feature_level=opensmile.FeatureLevel.Functionals  # Niveau des fonctionnalités (moyennes, std, etc.)
)

# Extraire les caractéristiques audio et les émotions associées aux fichiers
X = []
y = []

# Liste des fichiers audio dans le dossier
audio_files = sorted(os.listdir(audio_folder))

# Nombre total de fichiers
n_files = len(audio_files)

# Extraire les caractéristiques et les émotions
for i, audio_file in enumerate(audio_files):
    audio_path = os.path.join(audio_folder, audio_file)

    # Affichage de la progression
    if i % 100 == 0:  # Afficher toutes les 100 étapes
        print(f"Progression: {i}/{n_files} fichiers traités...")

    # Extraire les caractéristiques du fichier audio
    features = smile.process_file(audio_path)

    # Réinitialiser l'index pour enlever les niveaux multiples de MultiIndex
    features_reset = features.reset_index()

    # Convertir les caractéristiques en tableau NumPy
    features_array = features_reset.drop(['file', 'start', 'end'], axis=1).values

    # Ajouter les caractéristiques à la liste X
    X.append(features_array.flatten())  # Aplatir le tableau pour obtenir une ligne unique

    # Extraire l'émotion associée à ce fichier audio depuis le dataframe df_relevant
    emotion = df_relevant.iloc[i]['Emotion']

    # Ajouter l'émotion à la liste y après transformation en entier
    y.append(label_encoder.transform([emotion])[0])

# Convertir X et y en tableaux numpy
X = np.array(X)
y = np.array(y)

# Diviser les données en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Créer un modèle Random Forest
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Entraîner le modèle
print("Entraînement du modèle Random Forest...")
rf_classifier.fit(X_train, y_train)

# Faire des prédictions sur l'ensemble de test
y_pred = rf_classifier.predict(X_test)

# Afficher le rapport de classification
print("\nRapport de classification:")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

# Sauvegarder le modèle si nécessaire
import joblib
joblib.dump(rf_classifier, "random_forest_emotion_model.joblib")


Progression: 0/9988 fichiers traités...
Progression: 100/9988 fichiers traités...
Progression: 200/9988 fichiers traités...
Progression: 300/9988 fichiers traités...
Progression: 400/9988 fichiers traités...
Progression: 500/9988 fichiers traités...
Progression: 600/9988 fichiers traités...
Progression: 700/9988 fichiers traités...
Progression: 800/9988 fichiers traités...
Progression: 900/9988 fichiers traités...
Progression: 1000/9988 fichiers traités...
Progression: 1100/9988 fichiers traités...
Progression: 1200/9988 fichiers traités...
Progression: 1300/9988 fichiers traités...
Progression: 1400/9988 fichiers traités...
Progression: 1500/9988 fichiers traités...
Progression: 1600/9988 fichiers traités...
Progression: 1700/9988 fichiers traités...
Progression: 1800/9988 fichiers traités...
Progression: 1900/9988 fichiers traités...
Progression: 2000/9988 fichiers traités...
Progression: 2100/9988 fichiers traités...
Progression: 2200/9988 fichiers traités...
Progression: 2300/9988 

C:\Users\boucceredj\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\boucceredj\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\boucceredj\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


['random_forest_emotion_model.joblib']

In [3]:
!pip install xgboost

  Obtaining dependency information for xgboost from https://files.pythonhosted.org/packages/5e/03/15cd49e855c62226ecf1831bbe4c8e73a4324856077a23c495538a36e557/xgboost-3.0.0-py3-none-win_amd64.whl.metadata
   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.1/150.0 MB 2.0 MB/s eta 0:01:16
   ---------------------------------------- 0.1/150.0 MB 880.9 kB/s eta 0:02:51
   ---------------------------------------- 0.2/150.0 MB 1.1 MB/s eta 0:02:23
   ---------------------------------------- 0.2/150.0 MB 1.1 MB/s eta 0:02:23
   ---------------------------------------- 0.3/150.0 MB 927.4 kB/s eta 0:02:42
   ---------------------------------------- 0.3/150.0 MB 1.0 MB/s eta 0:02:25
   ---------------------------------------- 0.3/150.0 MB 1.0 MB/s eta 0:02:25
   ---------------------------------------- 0.4/150.0 MB 933.2 kB/s eta 0:02:41
   ---------------------------------------- 0.4/150.0 MB 804.0 kB/s eta 0:03:07
   -----------

In [2]:
import pandas as pd
import os
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

# Définir les chemins des fichiers
dataset_path = "dataset_clean"
train_csv = os.path.join(dataset_path, "train", "features_train_emotion.csv")
test_csv = os.path.join(dataset_path, "test", "features_test_emotion.csv")

# Charger les données
train_df = pd.read_csv(train_csv)
test_df = pd.read_csv(test_csv)

# Sélectionner les features et le label
feature_cols = [f"LDA_{i+1}" for i in range(6)]  # Les 6 features LDA
X_train = train_df[feature_cols].values
y_train = train_df["Emotion"].values

X_test = test_df[feature_cols].values
y_test = test_df["Emotion"].values

# Normalisation des features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Initialiser XGBoost
model = xgb.XGBClassifier(objective="multi:softmax", num_class=7, random_state=42)

# Recherche d'hyperparamètres
param_grid = {
    "n_estimators": [100, 300, 500],  # Nombre d'arbres
    "max_depth": [3, 6, 9],  # Profondeur des arbres
    "learning_rate": [0.01, 0.1, 0.3],  # Taux d’apprentissage
}

grid_search = GridSearchCV(model, param_grid, cv=3, scoring="accuracy", verbose=1)
grid_search.fit(X_train, y_train)

# Meilleur modèle
best_model = grid_search.best_estimator_

# Prédire sur les données de test
y_pred = best_model.predict(X_test)

# Évaluer la performance
accuracy = accuracy_score(y_test, y_pred)
print(f"Précision du modèle : {accuracy:.2f}")

# Afficher un rapport détaillé
print("\nRapport de classification :\n")
print(classification_report(y_test, y_pred))


ModuleNotFoundError: No module named 'xgboost'

In [5]:
import pandas as pd
import os
import numpy as np
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

# Charger les données
dataset_path = "dataset_clean"
train_csv = os.path.join(dataset_path, "train", "features_train_emotion.csv")
test_csv = os.path.join(dataset_path, "test", "features_test_emotion.csv")

train_df = pd.read_csv(train_csv)
test_df = pd.read_csv(test_csv)

# Sélectionner les features et le label
feature_cols = [f"LDA_{i+1}" for i in range(6)]  # Les 6 features LDA
X_train = train_df[feature_cols].values
y_train = train_df["Emotion"].values
X_test = test_df[feature_cols].values
y_test = test_df["Emotion"].values

# Liste des modèles à tester
models = {
    "Logistic Regression": LogisticRegression(max_iter=500),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(n_estimators=100, random_state=42),
    "KNN": KNeighborsClassifier(n_neighbors=5),
    "SVM": SVC(kernel='rbf', probability=True),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Naïve Bayes": GaussianNB(),
    "MLP (Neural Network)": MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, random_state=42),
    "AdaBoost": AdaBoostClassifier(n_estimators=100, random_state=42)
}

# Tester chaque modèle
best_model = None
best_accuracy = 0

for name, model in models.items():
    print(f"\n Test du modèle : {name}")

    # Entraînement
    model.fit(X_train, y_train)

    # Prédiction
    y_pred = model.predict(X_test)

    # Évaluation
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Précision : {accuracy:.4f}")

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model = (name, model)

    print("\nRapport de classification :\n")
    print(classification_report(y_test, y_pred))

# Afficher le meilleur modèle
print("\n Meilleur modèle : ", best_model[0])
print(f" Précision : {best_accuracy:.4f}")



 Test du modèle : Logistic Regression
Précision : 0.4617

Rapport de classification :

              precision    recall  f1-score   support

       anger       0.00      0.00      0.00       345
     disgust       1.00      0.75      0.86        68
        fear       1.00      0.42      0.59        50
         joy       0.00      0.00      0.00       402
     neutral       0.82      0.90      0.86      1256
     sadness       0.00      0.00      0.00       208
    surprise       0.01      0.01      0.01       281

    accuracy                           0.46      2610
   macro avg       0.40      0.30      0.33      2610
weighted avg       0.44      0.46      0.45      2610


 Test du modèle : Random Forest
Précision : 0.0211

Rapport de classification :

              precision    recall  f1-score   support

       anger       0.00      0.00      0.00       345
     disgust       0.00      0.00      0.00        68
        fear       0.00      0.00      0.00        50
         joy    

C:\Users\boucceredj\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\boucceredj\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\boucceredj\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

       anger       0.00      0.00      0.00       345
     disgust       0.98      0.85      0.91        68
        fear       0.00      0.00      0.00        50
         joy       0.00      0.00      0.00       402
     neutral       0.82      0.99      0.90      1256
     sadness       0.00      0.00      0.00       208
    surprise       0.04      0.06      0.05       281

    accuracy                           0.50      2610
   macro avg       0.26      0.27      0.27      2610
weighted avg       0.43      0.50      0.46      2610


 Test du modèle : SVM
Précision : 0.1533

Rapport de classification :

              precision    recall  f1-score   support

       anger       0.00      0.00      0.00       345
     disgust       0.00      0.00      0.00        68
        fear       0.00      0.00      0.00        50
         joy       0.16      1.00      0.27       402
     neutral       0.00      0.00      0.00      1256
     s

C:\Users\boucceredj\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\boucceredj\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\boucceredj\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Précision : 0.0019

Rapport de classification :

              precision    recall  f1-score   support

       anger       0.00      0.00      0.00       345
     disgust       0.00      0.00      0.00        68
        fear       0.00      0.00      0.00        50
         joy       0.00      0.00      0.00       402
     neutral       0.01      0.00      0.00      1256
     sadness       0.00      0.00      0.00       208
    surprise       0.00      0.01      0.01       281

    accuracy                           0.00      2610
   macro avg       0.00      0.00      0.00      2610
weighted avg       0.00      0.00      0.00      2610


 Test du modèle : Naïve Bayes
Précision : 0.6042

Rapport de classification :

              precision    recall  f1-score   support

       anger       0.00      0.00      0.00       345
     disgust       0.56      0.96      0.71        68
        fear       0.23      0.94      0.37        50
         joy       0.00      0.00      0.00       402
   

C:\Users\boucceredj\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\boucceredj\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\boucceredj\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Précision : 0.4042

Rapport de classification :

              precision    recall  f1-score   support

       anger       0.19      0.27      0.23       345
     disgust       0.00      0.00      0.00        68
        fear       0.00      0.00      0.00        50
         joy       0.00      0.00      0.00       402
     neutral       0.76      0.76      0.76      1256
     sadness       0.00      0.00      0.00       208
    surprise       0.00      0.00      0.00       281

    accuracy                           0.40      2610
   macro avg       0.14      0.15      0.14      2610
weighted avg       0.39      0.40      0.40      2610


 Meilleur modèle :  Naïve Bayes
 Précision : 0.6042


In [6]:
import pandas as pd
import os
import numpy as np
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler

# Charger les données
dataset_path = "dataset_clean"
train_csv = os.path.join(dataset_path, "train", "features_train_emotion.csv")
test_csv = os.path.join(dataset_path, "test", "features_test_emotion.csv")

train_df = pd.read_csv(train_csv)
test_df = pd.read_csv(test_csv)

# Sélectionner toutes les features sauf "Emotion"
feature_cols = [col for col in train_df.columns if col != "Emotion"]
X_train = train_df[feature_cols].values
y_train = train_df["Emotion"].values
X_test = test_df[feature_cols].values
y_test = test_df["Emotion"].values

# Normaliser les données
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Appliquer SMOTE pour équilibrer les classes
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

# Liste des modèles à tester
models = {
    "Logistic Regression": LogisticRegression(max_iter=500),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(n_estimators=100, random_state=42),
    "KNN": KNeighborsClassifier(n_neighbors=5),
    "SVM": SVC(kernel='rbf', probability=True),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Naïve Bayes": GaussianNB(),
    "MLP (Neural Network)": MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, random_state=42),
    "AdaBoost": AdaBoostClassifier(n_estimators=100, random_state=42)
}

# Tester chaque modèle
best_model = None
best_accuracy = 0

for name, model in models.items():
    print(f"\n Test du modèle : {name}")

    # Entraînement sur les données équilibrées
    model.fit(X_train_balanced, y_train_balanced)

    # Prédiction
    y_pred = model.predict(X_test)

    # Évaluation
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Précision : {accuracy:.4f}")

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model = (name, model)

    print("\nRapport de classification :\n")
    print(classification_report(y_test, y_pred))

# Afficher le meilleur modèle
print("\n Meilleur modèle : ", best_model[0])
print(f" Précision : {best_accuracy:.4f}")


ImportError: cannot import name '_MissingValues' from 'sklearn.utils._param_validation' (C:\Users\boucceredj\anaconda3\Lib\site-packages\sklearn\utils\_param_validation.py)

In [1]:
import pandas as pd
import os
import numpy as np
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

# Chemin vers les fichiers PCA
dataset_path = "dataset_clean"
train_csv = os.path.join(dataset_path, "train", "features_train_pca.csv")
test_csv = os.path.join(dataset_path, "test", "features_test_pca.csv")

# Chargement des données
train_df = pd.read_csv(train_csv)
test_df = pd.read_csv(test_csv)

# Colonnes PCA (2 composantes)
feature_cols = [f"PCA_{i+1}" for i in range(2)]
X_train = train_df[feature_cols].values
y_train = train_df["Emotion"].values
X_test = test_df[feature_cols].values
y_test = test_df["Emotion"].values

# Liste des modèles à tester
models = {
    "Logistic Regression": LogisticRegression(max_iter=500),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(n_estimators=100, random_state=42),
    "KNN": KNeighborsClassifier(n_neighbors=5),
    "SVM": SVC(kernel='rbf', probability=True),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Naïve Bayes": GaussianNB(),
    "MLP (Neural Network)": MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, random_state=42),
    "AdaBoost": AdaBoostClassifier(n_estimators=100, random_state=42)
}

# Tester les modèles
best_model = None
best_accuracy = 0

for name, model in models.items():
    print(f"\nTest du modèle : {name}")

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    print(f"Précision : {accuracy:.4f}")

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model = (name, model)

    print("\nRapport de classification :\n")
    print(classification_report(y_test, y_pred))

# Affichage du meilleur modèle
print("\nMeilleur modèle : ", best_model[0])
print(f"Précision : {best_accuracy:.4f}")



Test du modèle : Logistic Regression
Précision : 0.1172

Rapport de classification :

              precision    recall  f1-score   support

       anger       0.00      0.00      0.00       345
     disgust       0.00      0.00      0.00        68
        fear       0.00      0.00      0.00        50
         joy       0.16      0.16      0.16       402
     neutral       0.00      0.00      0.00      1256
     sadness       0.09      0.15      0.11       208
    surprise       0.11      0.75      0.20       281

    accuracy                           0.12      2610
   macro avg       0.05      0.15      0.07      2610
weighted avg       0.04      0.12      0.05      2610


Test du modèle : Random Forest


C:\Users\boucceredj\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\boucceredj\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\boucceredj\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Précision : 0.4126

Rapport de classification :

              precision    recall  f1-score   support

       anger       0.16      0.03      0.05       345
     disgust       0.00      0.00      0.00        68
        fear       0.00      0.00      0.00        50
         joy       0.16      0.08      0.11       402
     neutral       0.48      0.81      0.60      1256
     sadness       0.02      0.00      0.01       208
    surprise       0.11      0.06      0.08       281

    accuracy                           0.41      2610
   macro avg       0.13      0.14      0.12      2610
weighted avg       0.29      0.41      0.32      2610


Test du modèle : Gradient Boosting
Précision : 0.4069

Rapport de classification :

              precision    recall  f1-score   support

       anger       0.12      0.00      0.01       345
     disgust       0.00      0.00      0.00        68
        fear       0.00      0.00      0.00        50
         joy       0.22      0.13      0.16       40

C:\Users\boucceredj\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\boucceredj\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\boucceredj\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Précision : 0.2483

Rapport de classification :

              precision    recall  f1-score   support

       anger       0.13      0.09      0.11       345
     disgust       0.00      0.00      0.00        68
        fear       0.05      0.04      0.04        50
         joy       0.16      0.45      0.23       402
     neutral       0.47      0.32      0.39      1256
     sadness       0.06      0.02      0.03       208
    surprise       0.10      0.07      0.09       281

    accuracy                           0.25      2610
   macro avg       0.14      0.14      0.13      2610
weighted avg       0.29      0.25      0.25      2610


Test du modèle : Naïve Bayes
Précision : 0.4678

Rapport de classification :



C:\Users\boucceredj\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\boucceredj\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\boucceredj\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

       anger       0.17      0.01      0.01       345
     disgust       0.00      0.00      0.00        68
        fear       0.00      0.00      0.00        50
         joy       0.10      0.02      0.03       402
     neutral       0.48      0.96      0.64      1256
     sadness       0.00      0.00      0.00       208
    surprise       0.00      0.00      0.00       281

    accuracy                           0.47      2610
   macro avg       0.11      0.14      0.10      2610
weighted avg       0.27      0.47      0.31      2610


Test du modèle : MLP (Neural Network)
Précision : 0.3774

Rapport de classification :

              precision    recall  f1-score   support

       anger       0.00      0.00      0.00       345
     disgust       0.03      0.01      0.02        68
        fear       0.00      0.00      0.00        50
         joy       0.15      0.25      0.19       402
     neutral       0.47      0.70      0.56 

C:\Users\boucceredj\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\boucceredj\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\boucceredj\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Précision : 0.4812

Rapport de classification :

              precision    recall  f1-score   support

       anger       0.00      0.00      0.00       345
     disgust       0.00      0.00      0.00        68
        fear       0.00      0.00      0.00        50
         joy       0.00      0.00      0.00       402
     neutral       0.48      1.00      0.65      1256
     sadness       0.00      0.00      0.00       208
    surprise       0.00      0.00      0.00       281

    accuracy                           0.48      2610
   macro avg       0.07      0.14      0.09      2610
weighted avg       0.23      0.48      0.31      2610


Meilleur modèle :  SVM
Précision : 0.4812


C:\Users\boucceredj\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\boucceredj\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\boucceredj\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [3]:
!pip install librosa --user

In [ ]:
import os
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler
import collections

# 1. Chargement des CSVs générés par Librosa
dataset_path = "dataset_clean"
folders = ["train", "dev", "test"]

all_data = {}

for folder in folders:
    print(f"\n🔍 Chargement du fichier CSV pour le dossier : {folder}")
    csv_path = os.path.join(dataset_path, folder, f"features_{folder}_librosa.csv")
    
    try:
        # Charger les données du CSV dans un DataFrame
        df = pd.read_csv(csv_path)
        all_data[folder] = df
    except Exception as e:
        print(f"Erreur lors du chargement du fichier CSV {csv_path}: {e}")
        continue

# 2. Fusionner les données des différents dossiers
combined_df = pd.concat(all_data.values(), ignore_index=True)

# 3. Sélection des features MFCC et préparation des données
# Ici, tu utilises les colonnes MFCC_1 à MFCC_13
feature_cols = [f"MFCC_{i+1}" for i in range(13)]
X = combined_df[feature_cols].values
y = combined_df["Label"].values  # ou "Emotion" si tu veux uniquement l'émotion

# 4. Standardisation des données
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 5. Séparation train/test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, stratify=y, random_state=42)

# 6. Oversampling pour équilibrer les classes
ros = RandomOverSampler(random_state=42)
X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)

# Affichage de la répartition des classes avant et après oversampling
print("Avant oversampling:", collections.Counter(y_train))
print("Après oversampling:", collections.Counter(y_train_resampled))

# 7. Liste des modèles à tester
models = {
    "Logistic Regression": LogisticRegression(max_iter=500),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(n_estimators=100, random_state=42),
    "KNN": KNeighborsClassifier(n_neighbors=5),
    "SVM": SVC(kernel='rbf', probability=True),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Naïve Bayes": GaussianNB(),
    "MLP (Neural Network)": MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, random_state=42),
    "AdaBoost": AdaBoostClassifier(n_estimators=100, random_state=42),
}

# 8. Tester les modèles
best_model = None
best_accuracy = 0

for name, model in models.items():
    print(f"\n🔍 Test du modèle : {name}")
    
    # Entraîner le modèle sur les données oversamplées
    model.fit(X_train_resampled, y_train_resampled)
    y_pred = model.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    print(f"🎯 Précision : {acc:.4f}")

    if acc > best_accuracy:
        best_accuracy = acc
        best_model = (name, model)

    print("📊 Rapport de classification :\n")
    print(classification_report(y_test, y_pred))

# 9. Résumé
print("\n✅ Meilleur modèle :", best_model[0])
print(f"🎯 Meilleure précision : {best_accuracy:.4f}")



🔍 Chargement du fichier CSV pour le dossier : train

🔍 Chargement du fichier CSV pour le dossier : dev

🔍 Chargement du fichier CSV pour le dossier : test
Avant oversampling: Counter({'neutral_neutral': 5147, 'joy_positive': 1846, 'anger_negative': 1285, 'sadness_negative': 802, 'surprise_negative': 685, 'surprise_positive': 624, 'disgust_negative': 289, 'fear_negative': 286})
Après oversampling: Counter({'neutral_neutral': 5147, 'surprise_positive': 5147, 'sadness_negative': 5147, 'anger_negative': 5147, 'joy_positive': 5147, 'surprise_negative': 5147, 'disgust_negative': 5147, 'fear_negative': 5147})

🔍 Test du modèle : Logistic Regression
🎯 Précision : 0.2203
📊 Rapport de classification :

                   precision    recall  f1-score   support

   anger_negative       0.24      0.32      0.28       322
 disgust_negative       0.05      0.25      0.09        72
    fear_negative       0.01      0.01      0.01        72
     joy_positive       0.18      0.13      0.15       462
 

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

# Charger les données à partir des CSV
df_train = pd.read_csv("dataset_clean/train/features_train_librosa.csv")
df_dev = pd.read_csv("dataset_clean/dev/features_dev_librosa.csv")
df_test = pd.read_csv("dataset_clean/test/features_test_librosa.csv")

# Fusionner les données
combined_df = pd.concat([df_train, df_dev, df_test], ignore_index=True)

# Sélectionner les features et les labels
feature_cols = [f"MFCC_{i+1}" for i in range(13)] + [f"Chroma_{i+1}" for i in range(12)] + ["Spectral_Rolloff", "Zero_Crossing_Rate", "RMSE"]
X = combined_df[feature_cols].values
y = combined_df["Label"].values  # Ou "Emotion" ou "Sentiment" si tu veux prédire des catégories spécifiques

# Encoder les labels (si c'est nécessaire)
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Normaliser les caractéristiques
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Diviser en train et test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, stratify=y_encoded, random_state=42)

# Reshaping des données pour le CNN (si tu veux utiliser un CNN sur les spectrogrammes, par exemple)
# Ici, nous supposons que les données sont des vecteurs (features), donc un reshape peut ne pas être nécessaire pour un MLP ou SVM
# Mais si tu choisis d'utiliser un CNN, tu devras convertir chaque feature vector en une forme 2D ou 3D.
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)  # Si tu choisis un CNN

# 3. Construire un modèle TensorFlow (par exemple un MLP ou CNN)

# Exemple : Construire un modèle MLP simple pour cette tâche
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(len(np.unique(y)), activation='softmax')  # Classification multiclasse
])

# Compiler le modèle
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Entraîner le modèle
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluation
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy:.4f}")


C:\Users\boucceredj\anaconda3\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
343/343 ━━━━━━━━━━━━━━━━━━━━ 15s 14ms/step - accuracy: 0.4398 - loss: 1.7027 - val_accuracy: 0.4694 - val_loss: 1.6097
Epoch 2/10
343/343 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.4828 - loss: 1.5753 - val_accuracy: 0.4730 - val_loss: 1.5911
Epoch 3/10
343/343 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.4678 - loss: 1.5975 - val_accuracy: 0.4708 - val_loss: 1.5888
Epoch 4/10
343/343 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.4814 - loss: 1.5534 - val_accuracy: 0.4690 - val_loss: 1.5880
Epoch 5/10
343/343 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.4666 - loss: 1.5646 - val_accuracy: 0.4712 - val_loss: 1.5849
Epoch 6/10
343/343 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4821 - loss: 1.5468 - val_accuracy: 0.4697 - val_loss: 1.5784
Epoch 7/10
343/343 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4755 - loss: 1.5431 - val_accuracy: 0.4694 - val_loss: 1.5779
Epoch 8/10
343/343 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4702 - loss: 1.5411 - val_accuracy

In [1]:
import os
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler
import collections

# 1. Chargement des CSVs générés par Librosa
dataset_path = "dataset_clean"
folders = ["train", "dev", "test"]

all_data = {}

for folder in folders:
    print(f"\n🔍 Chargement du fichier CSV pour le dossier : {folder}")
    csv_path = os.path.join(dataset_path, folder, f"features_{folder}_librosa.csv")
    
    try:
        df = pd.read_csv(csv_path)
        all_data[folder] = df
    except Exception as e:
        print(f"Erreur lors du chargement du fichier CSV {csv_path}: {e}")
        continue

# 2. Fusionner les données des différents dossiers
combined_df = pd.concat(all_data.values(), ignore_index=True)

# 3. Sélection des features MFCC (tu peux aussi ajouter Chroma ou autres si tu veux)
feature_cols = [f"MFCC_{i+1}" for i in range(13)]
X = combined_df[feature_cols].values

# Fonction pour entraîner et évaluer un modèle sur une cible donnée
def train_evaluate_model(y_label):
    print(f"\n======= Modèle pour la variable cible : {y_label} =======")
    y = combined_df[y_label].values

    # 4. Standardisation des données
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # 5. Séparation train/test (stratifiée)
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, stratify=y, random_state=42)

    # 6. Oversampling pour équilibrer les classes
    ros = RandomOverSampler(random_state=42)
    X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)

    print(f"Avant oversampling : {collections.Counter(y_train)}")
    print(f"Après oversampling : {collections.Counter(y_train_resampled)}")

    # 7. Modèles à tester (tu peux changer / ajouter)
    models = {
        "Logistic Regression": LogisticRegression(max_iter=500),
        "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
        "Gradient Boosting": GradientBoostingClassifier(n_estimators=100, random_state=42),
        "KNN": KNeighborsClassifier(n_neighbors=5),
        #"SVM": SVC(kernel='rbf', probability=True),
        "Decision Tree": DecisionTreeClassifier(random_state=42),
        "Naïve Bayes": GaussianNB(),
        "MLP (Neural Network)": MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, random_state=42),
        "AdaBoost": AdaBoostClassifier(n_estimators=100, random_state=42),
    }

    best_model = None
    best_accuracy = 0

    for name, model in models.items():
        print(f"\n🔍 Test du modèle : {name}")
        model.fit(X_train_resampled, y_train_resampled)
        y_pred = model.predict(X_test)

        acc = accuracy_score(y_test, y_pred)
        print(f"🎯 Précision : {acc:.4f}")

        if acc > best_accuracy:
            best_accuracy = acc
            best_model = (name, model)

        print("📊 Rapport de classification :\n")
        print(classification_report(y_test, y_pred))

    print(f"\n✅ Meilleur modèle pour {y_label} : {best_model[0]}")
    print(f"🎯 Meilleure précision : {best_accuracy:.4f}")

# Entraînement et évaluation pour Emotion
train_evaluate_model("Emotion")

# Entraînement et évaluation pour Sentiment
train_evaluate_model("Sentiment")



🔍 Chargement du fichier CSV pour le dossier : train

🔍 Chargement du fichier CSV pour le dossier : dev

🔍 Chargement du fichier CSV pour le dossier : test

======= Modèle pour la variable cible : Emotion =======
Avant oversampling : Counter({'neutral': 5147, 'joy': 1846, 'surprise': 1309, 'anger': 1285, 'sadness': 802, 'disgust': 289, 'fear': 286})
Après oversampling : Counter({'neutral': 5147, 'anger': 5147, 'sadness': 5147, 'joy': 5147, 'surprise': 5147, 'disgust': 5147, 'fear': 5147})

🔍 Test du modèle : Logistic Regression
🎯 Précision : 0.2327
📊 Rapport de classification :

              precision    recall  f1-score   support

       anger       0.24      0.34      0.28       322
     disgust       0.06      0.31      0.10        72
        fear       0.02      0.04      0.03        72
         joy       0.18      0.14      0.16       462
     neutral       0.58      0.25      0.35      1287
     sadness       0.11      0.32      0.17       200
    surprise       0.18      0.16  

C:\Users\boucceredj\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


🎯 Précision : 0.1864
📊 Rapport de classification :

              precision    recall  f1-score   support

       anger       0.17      0.26      0.21       322
     disgust       0.04      0.19      0.07        72
        fear       0.04      0.14      0.06        72
         joy       0.18      0.13      0.15       462
     neutral       0.56      0.18      0.28      1287
     sadness       0.09      0.23      0.13       200
    surprise       0.16      0.20      0.17       327

    accuracy                           0.19      2742
   macro avg       0.18      0.19      0.15      2742
weighted avg       0.34      0.19      0.21      2742


🔍 Test du modèle : AdaBoost
🎯 Précision : 0.2250
📊 Rapport de classification :

              precision    recall  f1-score   support

       anger       0.22      0.24      0.23       322
     disgust       0.04      0.19      0.07        72
        fear       0.04      0.14      0.06        72
         joy       0.20      0.15      0.17       462

In [2]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout, Masking, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import StandardScaler

# ==== Chargement des données comme dans ton code ====
# On suppose que combined_df est déjà construit avec tous les MFCCs + Emotion + Sentiment
feature_cols = [f"MFCC_{i+1}" for i in range(13)]
X = combined_df[feature_cols].values
X = StandardScaler().fit_transform(X)

# S'il s'agit de séquences MFCC temporelles, forme en 3D (sinon saute cette étape)
# Par exemple: reshape en (samples, timesteps=13, features=1) juste pour simulation
X = X.reshape(-1, 13, 1)  # ou adapte selon ta vraie structure temporelle

def train_deep_model(target_col, cell_type="LSTM"):
    print(f"\n🧠 Modèle Deep Learning pour {target_col} avec {cell_type}")

    y_raw = combined_df[target_col].values

    # Encodage des classes
    le = LabelEncoder()
    y_encoded = le.fit_transform(y_raw)
    y_cat = to_categorical(y_encoded)

    # Train/test split
    X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.2, stratify=y_encoded, random_state=42)

    # Oversampling uniquement si les classes sont déséquilibrées (sur les labels, pas one-hot)
    ros = RandomOverSampler(random_state=42)
    X_train_flat = X_train.reshape((X_train.shape[0], -1))  # nécessaire pour ROS
    y_train_int = np.argmax(y_train, axis=1)

    X_train_flat, y_train_int = ros.fit_resample(X_train_flat, y_train_int)

    X_train = X_train_flat.reshape((-1, X.shape[1], X.shape[2]))  # reshape après ROS
    y_train = to_categorical(y_train_int)

    # Construction du modèle
    model = Sequential()
    model.add(Masking(mask_value=0., input_shape=(X.shape[1], X.shape[2])))

    if cell_type == "LSTM":
        model.add(LSTM(128, return_sequences=True))
        model.add(BatchNormalization())
        model.add(Dropout(0.3))
        model.add(LSTM(128))
    elif cell_type == "GRU":
        model.add(GRU(128, return_sequences=True))
        model.add(BatchNormalization())
        model.add(Dropout(0.3))
        model.add(GRU(128))
    else:
        raise ValueError("cell_type doit être 'LSTM' ou 'GRU'.")

    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(y_cat.shape[1], activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Early stopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)

    # Entraînement
    history = model.fit(
        X_train, y_train,
        epochs=50,
        batch_size=64,
        validation_split=0.2,
        callbacks=[early_stopping],
        verbose=2
    )

    # Évaluation
    loss, acc = model.evaluate(X_test, y_test, verbose=0)
    print(f"\n✅ Accuracy sur le jeu de test ({target_col}) : {acc:.4f}")

# ==== Lancer pour Emotion et Sentiment ====
train_deep_model("Emotion", cell_type="LSTM")
train_deep_model("Sentiment", cell_type="GRU")



🧠 Modèle Deep Learning pour Emotion avec LSTM


C:\Users\boucceredj\anaconda3\Lib\site-packages\keras\src\layers\core\masking.py:47: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
451/451 - 42s - 94ms/step - accuracy: 0.1955 - loss: 1.9970 - val_accuracy: 0.0000e+00 - val_loss: 2.4207
Epoch 2/50
451/451 - 40s - 89ms/step - accuracy: 0.2113 - loss: 1.8647 - val_accuracy: 0.0000e+00 - val_loss: 2.7025
Epoch 3/50
451/451 - 20s - 43ms/step - accuracy: 0.2243 - loss: 1.8414 - val_accuracy: 0.0000e+00 - val_loss: 2.6744
Epoch 4/50
451/451 - 19s - 43ms/step - accuracy: 0.2326 - loss: 1.8279 - val_accuracy: 0.0000e+00 - val_loss: 2.7200
Epoch 5/50
451/451 - 20s - 44ms/step - accuracy: 0.2435 - loss: 1.8141 - val_accuracy: 0.0000e+00 - val_loss: 2.8255
Epoch 6/50
451/451 - 19s - 43ms/step - accuracy: 0.2549 - loss: 1.7977 - val_accuracy: 0.0000e+00 - val_loss: 2.6915
Epoch 7/50
451/451 - 19s - 43ms/step - accuracy: 0.2754 - loss: 1.7724 - val_accuracy: 0.0000e+00 - val_loss: 2.7641
Epoch 8/50
451/451 - 19s - 43ms/step - accuracy: 0.2998 - loss: 1.7320 - val_accuracy: 0.0000e+00 - val_loss: 2.7035

✅ Accuracy sur le jeu de test (Emotion) : 0.1258

🧠 Modèle Deep

C:\Users\boucceredj\anaconda3\Lib\site-packages\keras\src\layers\core\masking.py:47: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
193/193 - 32s - 166ms/step - accuracy: 0.3925 - loss: 1.2265 - val_accuracy: 0.0715 - val_loss: 1.2912
Epoch 2/50
193/193 - 19s - 101ms/step - accuracy: 0.4231 - loss: 1.0988 - val_accuracy: 0.0881 - val_loss: 1.3795
Epoch 3/50
193/193 - 6s - 33ms/step - accuracy: 0.4312 - loss: 1.0732 - val_accuracy: 0.0800 - val_loss: 1.3630
Epoch 4/50
193/193 - 7s - 36ms/step - accuracy: 0.4472 - loss: 1.0593 - val_accuracy: 0.0304 - val_loss: 1.4252
Epoch 5/50
193/193 - 9s - 45ms/step - accuracy: 0.4516 - loss: 1.0549 - val_accuracy: 0.0489 - val_loss: 1.3866
Epoch 6/50
193/193 - 14s - 75ms/step - accuracy: 0.4515 - loss: 1.0525 - val_accuracy: 0.0725 - val_loss: 1.4757
Epoch 7/50
193/193 - 10s - 50ms/step - accuracy: 0.4577 - loss: 1.0482 - val_accuracy: 0.0285 - val_loss: 1.4616
Epoch 8/50
193/193 - 9s - 49ms/step - accuracy: 0.4598 - loss: 1.0469 - val_accuracy: 0.0754 - val_loss: 1.4403

✅ Accuracy sur le jeu de test (Sentiment) : 0.4278


In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import pandas as pd
import numpy as np

# Charger les données
df = pd.read_csv("dataset_clean/train/features_train_librosa.csv")  # à adapter
X = df[[f"MFCC_{i+1}" for i in range(13)]].values
y_emotion = df["Emotion"].values
y_sentiment = df["Sentiment"].values

# Prétraitement
scaler = StandardScaler()
X = scaler.fit_transform(X)
X = X.reshape((X.shape[0], X.shape[1], 1))

le_emotion = LabelEncoder()
y_emotion_enc = le_emotion.fit_transform(y_emotion)

le_sentiment = LabelEncoder()
y_sentiment_enc = le_sentiment.fit_transform(y_sentiment)

# Split
X_train, X_test, y_train_emotion, y_test_emotion = train_test_split(X, y_emotion_enc, test_size=0.2, random_state=42, stratify=y_emotion_enc)
_, _, y_train_sentiment, y_test_sentiment = train_test_split(X, y_sentiment_enc, test_size=0.2, random_state=42, stratify=y_sentiment_enc)

# Modèle CNN + LSTM
def build_model(num_classes):
    model = Sequential([
        Input(shape=(13, 1)),
        Conv1D(64, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        BatchNormalization(),
        LSTM(64),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Emotion
model_emotion = build_model(num_classes=len(np.unique(y_emotion_enc)))
model_emotion.fit(X_train, y_train_emotion, epochs=20, batch_size=32, validation_split=0.2,
                  callbacks=[EarlyStopping(patience=3, restore_best_weights=True)], verbose=2)
emotion_acc = model_emotion.evaluate(X_test, y_test_emotion, verbose=0)[1]
print(f"🎯 Emotion Accuracy : {emotion_acc:.4f}")

# Sentiment
model_sentiment = build_model(num_classes=len(np.unique(y_sentiment_enc)))
model_sentiment.fit(X_train, y_train_sentiment, epochs=20, batch_size=32, validation_split=0.2,
                    callbacks=[EarlyStopping(patience=3, restore_best_weights=True)], verbose=2)
sentiment_acc = model_sentiment.evaluate(X_test, y_test_sentiment, verbose=0)[1]
print(f"🎯 Sentiment Accuracy : {sentiment_acc:.4f}")


Epoch 1/20
200/200 - 20s - 100ms/step - accuracy: 0.4416 - loss: 1.6182 - val_accuracy: 0.4756 - val_loss: 1.5986
Epoch 2/20
200/200 - 2s - 12ms/step - accuracy: 0.4667 - loss: 1.5516 - val_accuracy: 0.4706 - val_loss: 1.5471
Epoch 3/20
200/200 - 2s - 9ms/step - accuracy: 0.4693 - loss: 1.5396 - val_accuracy: 0.4756 - val_loss: 1.5336
Epoch 4/20
200/200 - 2s - 8ms/step - accuracy: 0.4707 - loss: 1.5271 - val_accuracy: 0.4756 - val_loss: 1.5263
Epoch 5/20
200/200 - 2s - 10ms/step - accuracy: 0.4692 - loss: 1.5260 - val_accuracy: 0.4737 - val_loss: 1.5261
Epoch 6/20
200/200 - 2s - 9ms/step - accuracy: 0.4712 - loss: 1.5161 - val_accuracy: 0.4743 - val_loss: 1.5296
Epoch 7/20
200/200 - 2s - 8ms/step - accuracy: 0.4709 - loss: 1.5086 - val_accuracy: 0.4756 - val_loss: 1.5272
Epoch 8/20
200/200 - 2s - 8ms/step - accuracy: 0.4700 - loss: 1.5071 - val_accuracy: 0.4750 - val_loss: 1.5269
🎯 Emotion Accuracy : 0.4735
Epoch 1/20
200/200 - 19s - 94ms/step - accuracy: 0.4553 - loss: 1.0715 - val_ac